In [2]:
import resource
import time

RESOURCES = [
    ('ru_utime', 'User time'),
    ('ru_stime', 'System time'),
    ('ru_maxrss', 'Max. Resident Set Size'),
    ('ru_ixrss', 'Shared Memory Size'),
    ('ru_idrss', 'Unshared Memory Size'),
    ('ru_isrss', 'Stack Size'),
    ('ru_inblock', 'Block inputs'),
    ('ru_oublock', 'Block outputs'),
]
# 使用 getrusage() 来探测当前进程和/或其子进程使用的资源。 返回值是一个数据结构，包含基于系统当前状态的多个资源指标。
usage = resource.getrusage(resource.RUSAGE_SELF)

for name, desc in RESOURCES:
    print(f"{desc :>25} ({name:<10}) = {getattr(usage, name)}")

                User time (ru_utime  ) = 0.406485
              System time (ru_stime  ) = 0.040246
   Max. Resident Set Size (ru_maxrss ) = 48224
       Shared Memory Size (ru_ixrss  ) = 0
     Unshared Memory Size (ru_idrss  ) = 0
               Stack Size (ru_isrss  ) = 0
             Block inputs (ru_inblock) = 0
            Block outputs (ru_oublock) = 128


In [3]:
LIMITS = [
    ('RLIMIT_CORE', 'core file size'),
    ('RLIMIT_CPU', 'CPU time'),
    ('RLIMIT_FSIZE', 'file size'),
    ('RLIMIT_DATA', 'heap size'),
    ('RLIMIT_STACK', 'stack size'),
    ('RLIMIT_RSS', 'resident set size'),
    ('RLIMIT_NPROC', 'number of processes'),
    ('RLIMIT_NOFILE', 'number of open files'),
    ('RLIMIT_MEMLOCK', 'lockable memory address'),
]

for name, desc in LIMITS:
    limit_num = getattr(resource, name)
    # 每个限制的返回值是一个元组，其中包含当前配置强加的 软 限制和操作系统强加的 硬 限制。
    soft, hard = resource.getrlimit(limit_num)
    print(f"{desc:<23} {soft}/{hard}")

core file size          0/-1
CPU time                -1/-1
file size               -1/-1
heap size               -1/-1
stack size              8388608/-1
resident set size       -1/-1
number of processes     31405/31405
number of open files    1024/4096
lockable memory address 16777216/16777216


In [8]:
soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
print("soft limit starts as", soft)

resource.setrlimit(resource.RLIMIT_NOFILE, (100, hard))

soft, hard = resource.getrlimit(resource.RLIMIT_NOFILE)
print("soft limit sets to", soft)
print(hard)

soft limit starts as 1024
soft limit sets to 100
4096


In [1]:
import sys
import signal
import time
import resource

# 设置一个信号处理程序，以便在我们用完时通知我们
def time_expired(n, stack):
    print('expired', time.ctime())
    raise SystemExit('(time ran out)')
    
signal.signal(signal.SIGXCPU, time_expired)

soft, hard = resource.getrlimit(resource.RLIMIT_CPU)
print("soft limit starts as", soft)

# 调整CPU时间限制
resource.setrlimit(resource.RLIMIT_CPU, (1, hard))

soft, hard = resource.getrlimit(resource.RLIMIT_CPU)
print("soft limit sets to", soft)
print(hard)


print('starting:', time.ctime())

# 消耗CPU时间
for i in range(200000):
    for i in range(200000):
        v = i*i

# 永远也运行不到这里
print('Exiting:', time.ctime())        

soft limit starts as -1
soft limit sets to 1
-1
starting: Sat Oct 27 11:04:36 2018
expired Sat Oct 27 11:04:37 2018


SystemExit: (time ran out)

/home/hejl/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3273: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
